In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/VRD-IU

/content/drive/MyDrive/VRD-IU


In [3]:
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import os
import torchvision.transforms as transforms
import pickle
from transformers import AutoImageProcessor
import torch
class CompVisualDataset(Dataset):
    def __init__(self, pickle_file,image_path_root):
        super().__init__()
        with open(pickle_file, 'rb') as file:
            data = pickle.load(file)
        self.components = []
        self.image_paths = []
        for k in data.keys():
            self.components.extend(data[k]['components'])
            self.image_paths.extend([ f"{os.path.join(image_path_root,k)}_page-{comp['page']}.png" for comp in data[k]['components']])


    def __len__(self):
        return len(self.components)

    def __getitem__(self, index):
        comp = self.components[index]
        img = Image.open(self.image_paths[index]).convert("RGB")
        bbox = comp['bbox']
        cropped_img = transforms.functional.crop(img,top=bbox[1],left=bbox[0],height=bbox[3],width=bbox[2])
        return cropped_img, comp['object_id']

def collate_fn(batch):
    imgs = [e[0] for e in batch]
    object_ids = [e[1] for e in batch]
    return imgs, object_ids

In [4]:
train_dataset = CompVisualDataset('train_data.pkl','train/train')
train_dataloader = DataLoader(train_dataset,batch_size=256, collate_fn= collate_fn, num_workers=2)
val_dataset = CompVisualDataset('val_data.pkl','val/val')
val_dataloader = DataLoader(val_dataset,batch_size=256, collate_fn= collate_fn, num_workers=2)
test_dataset = CompVisualDataset('test_data.pkl','test/test')
test_dataloader = DataLoader(test_dataset,batch_size=256, collate_fn= collate_fn, num_workers=2)

In [5]:
from transformers import  DonutSwinModel
model = DonutSwinModel.from_pretrained("./donut_encoder")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using: {device}")
model.to(device)
model.eval()

Using: cuda


DonutSwinModel(
  (embeddings): DonutSwinEmbeddings(
    (patch_embeddings): DonutSwinPatchEmbeddings(
      (projection): Conv2d(3, 128, kernel_size=(4, 4), stride=(4, 4))
    )
    (norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.0, inplace=False)
  )
  (encoder): DonutSwinEncoder(
    (layers): ModuleList(
      (0): DonutSwinStage(
        (blocks): ModuleList(
          (0): DonutSwinLayer(
            (layernorm_before): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
            (attention): DonutSwinAttention(
              (self): DonutSwinSelfAttention(
                (query): Linear(in_features=128, out_features=128, bias=True)
                (key): Linear(in_features=128, out_features=128, bias=True)
                (value): Linear(in_features=128, out_features=128, bias=True)
                (dropout): Dropout(p=0.0, inplace=False)
              )
              (output): DonutSwinSelfOutput(
                (dense): Linear(in

In [6]:
image_processor = AutoImageProcessor.from_pretrained("nielsr/donut-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/362 [00:00<?, ?B/s]

In [7]:
from tqdm import tqdm
import os
def extract_features(dataloader, feature_path):
  if not os.path.exists(feature_path):
    os.makedirs(feature_path)
  with torch.no_grad():
      for imgs, object_ids in tqdm(dataloader):
          image_inputs = image_processor(imgs, return_tensors="pt").to(device)
          outputs = model(**image_inputs)
          features = outputs.pooler_output.detach().cpu()
          for idx, obj_id in enumerate(object_ids):
            torch.save(features[idx],os.path.join(feature_path,f"{obj_id}.pt"))

In [ ]:
extract_features(train_dataloader,  'train_visual_features')
print("Extraction completed for training set!")
extract_features(val_dataloader,  'val_visual_features')
print("Extraction completed for validation set!")
extract_features(test_dataloader,  'test_visual_features')
print("Extraction completed for testing set!")

  0%|          | 0/170 [00:00<?, ?it/s]